In [1]:
import csv
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Note 1 : 2018 and after adds a column called "Strength of Schedule" which changes the index of "Win division", "Clinch wildcard", and "make playoffs". There is a need to load both datasets separately and then merge them. 
Note 2: 2022 and after has a new column "clinch bye". 
Note 3: Possible to speed this up by interacting with the calender menu and selecting a day. 

https://www.fangraphs.com/standings/playoff-odds/fg/wc?date=2014-03-31&dateDelta=

In [1]:
chrome_options = Options()
chrome_options.add_argument("--headless")
service = Service("L:/chromedriver/chromedriver-win64/chromedriver.exe")

NameError: name 'Options' is not defined

In [2]:
from datetime import datetime, timedelta


start_date = datetime(2014, 3, 30)

end_date = datetime.today()

date_list = []



current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)

print(date_list.index("2020-03-11"))
url_wc = ["https://www.fangraphs.com/standings/playoff-odds/fg/wc?date=" + x + "&dateDelta=" for x in date_list]
url_lg = ["https://www.fangraphs.com/standings/playoff-odds/fg/lg?date=" + x + "&dateDelta=" for x in date_list]

2173


In [4]:
# all the names of the teams + the headers for the csv
mlb_teams = [
    "Diamondbacks","Braves", "Orioles", "Red Sox", "White Sox", "Cubs",
    "Reds", "Guardians", "Rockies", "Tigers", "Astros", "Royals", 
    "Angels", "Dodgers", "Marlins", "Brewers", "Twins", "Mets", 
    "Yankees", "Athletics", "Phillies", "Pirates", "Padres", "Giants", 
    "Mariners", "Cardinals", "Rays", "Rangers", "Blue Jays", "Nationals"
]

headers = ["year", "team", "game", "date", "win", "loss", "GB", "GBWC",
           "win division", "clinch wildcard", "make playoffs"]



In [5]:
def parse_html(table, is_wildcard: bool) -> list: 
    """Parses through the html given."""
    res = []
    soup = BeautifulSoup(table, 'html.parser')
    rows = soup.find('tbody').find_all('tr')

    for row in rows:
        info = row.find_all('td')
        if is_wildcard:
            name  = info[0].find('span', {'class': 'fullName'}).text.strip()
            if "+" in info[4].text.strip():
                res.append((name, "up " + info[4].text.strip()[1:]))
            else:
                res.append((name, info[4].text.strip()))
        else:
            team_info = [' ' for _ in range(11)]
            for index, col in enumerate(info):
                if index == 0: # name
                    team_name = row.find('span', {'class': 'fullName'}).text.strip()
                    team_info[1] = team_name
                elif index == 1 or index == 2: # wins and losses 
                    team_info[index + 3] = col.text.strip()
                elif index == 4: # gb
                    team_info[6] = col.text.strip() 
                    
                elif index == 8: # win division 
                    team_info[8] = col.text.strip()
                    
                elif index == 9: # clinch wildcard
                    team_info[9] = col.text.strip()
                    
                elif index == 10: # make playoffs
                    team_info[10] = col.text.strip()
            res.append(team_info)
    return res



In [6]:
def parse_html_2018(table, is_wildcard: bool) -> list: 
    """Parses through the html given."""
    res = []
    soup = BeautifulSoup(table, 'html.parser')
    rows = soup.find('tbody').find_all('tr')

    for row in rows:
        info = row.find_all('td')
        if is_wildcard:
            name  = info[0].find('span', {'class': 'fullName'}).text.strip()
            if "+" in info[4].text.strip():
                res.append((name, "up " + info[4].text.strip()[1:]))
            else:
                res.append((name, info[4].text.strip()))
        else:
            team_info = [' ' for _ in range(11)]
            for index, col in enumerate(info):
                if index == 0: # name
                    team_name = row.find('span', {'class': 'fullName'}).text.strip()
                    team_info[1] = team_name
                elif index == 1 or index == 2: # wins and losses 
                    team_info[index + 3] = col.text.strip()
                elif index == 4: # gb
                    team_info[6] = col.text.strip() 
                    
                elif index == 9: # win division 
                    team_info[8] = col.text.strip()
                    
                elif index == 10: # clinch wildcard
                    team_info[9] = col.text.strip()
                    
                elif index == 11: # make playoffs
                    team_info[10] = col.text.strip()
            res.append(team_info)
    return res



In [7]:
def parse_html_2022(table, is_wildcard: bool) -> list: 
    """Parses through the html given."""
    res = []
    soup = BeautifulSoup(table, 'html.parser')
    rows = soup.find('tbody').find_all('tr')

    for row in rows:
        info = row.find_all('td')
        if is_wildcard:
            name  = info[0].find('span', {'class': 'fullName'}).text.strip()
            if "+" in info[4].text.strip():
                res.append((name, "up " + info[4].text.strip()[1:]))
            else:
                res.append((name, info[4].text.strip()))
        else:
            team_info = [' ' for _ in range(11)]
            for index, col in enumerate(info):
                if index == 0: # name
                    team_name = row.find('span', {'class': 'fullName'}).text.strip()
                    team_info[1] = team_name
                elif index == 1 or index == 2: # wins and losses 
                    team_info[index + 3] = col.text.strip()
                elif index == 4: # gb
                    team_info[6] = col.text.strip() 
                    
                elif index == 9: # win division 
                    team_info[8] = col.text.strip()
                    
                elif index == 11: # clinch wildcard
                    team_info[9] = col.text.strip()
                    
                elif index == 12: # make playoffs
                    team_info[10] = col.text.strip()
            res.append(team_info)
    return res



In [8]:
def combine(lg, wc, date):
    """Combines the data gathered from wildcard and league."""
    
    for team in lg:
        games = int(lg[team][4]) + int(lg[team][5]) + 1
        lg[team][0] = date[:4]
        lg[team][2] = games
        lg[team][3] = date
        
        if team in wc:
            lg[team][7] = wc[team][1]
    return lg
    

In [9]:
def load_page(driver, is_wildcard: bool, year) -> dict:
    """Loads the page given url."""
    try:
        WebDriverWait(driver, 8).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'playoff-odds-table')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_html = soup.find_all('table', 'playoff-odds-table')
        AL_table_html = str(table_html[1])
        NL_table_html = str(table_html[3])

        if year == "2017":
            AL_table = parse_html(AL_table_html, is_wildcard)
            NL_table = parse_html(NL_table_html, is_wildcard)
        elif year == "2018":
            AL_table = parse_html_2018(AL_table_html, is_wildcard)
            NL_table = parse_html_2018(NL_table_html, is_wildcard)
        elif year == "2022":
            AL_table = parse_html_2022(AL_table_html, is_wildcard)
            NL_table = parse_html_2022(NL_table_html, is_wildcard)
        total = AL_table + NL_table
        if is_wildcard:
            res = {x[0]: x for x in total}
        else:
            res = {x[1]: x for x in total}
        
        
        return res
    except:
        raise ValueError
    


In [10]:
"""testing"""
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url_lg[2929])
lg_data = load_page(driver, False, "2022")
driver.get(url_wc[2929])
wc_data = load_page(driver, True, "2022")
print(lg_data)
print(date_list[2929])

{'Blue Jays': [' ', 'Blue Jays', ' ', ' ', '0', '0', '0.0', ' ', '43.8%', '45.8%', '89.6%'], 'Yankees': [' ', 'Yankees', ' ', ' ', '0', '0', '0.0', ' ', '34.6%', '51.2%', '85.8%'], 'Astros': [' ', 'Astros', ' ', ' ', '0', '0', '0.0', ' ', '71.3%', '14.4%', '85.7%'], 'White Sox': [' ', 'White Sox', ' ', ' ', '0', '0', '0.0', ' ', '59.2%', '12.9%', '72.2%'], 'Red Sox': [' ', 'Red Sox', ' ', ' ', '0', '0', '0.0', ' ', '12.9%', '48.4%', '61.3%'], 'Rays': [' ', 'Rays', ' ', ' ', '0', '0', '0.0', ' ', '8.7%', '44.1%', '52.7%'], 'Angels': [' ', 'Angels', ' ', ' ', '0', '0', '0.0', ' ', '18.5%', '26.2%', '44.7%'], 'Twins': [' ', 'Twins', ' ', ' ', '0', '0', '0.0', ' ', '23.3%', '17.2%', '40.4%'], 'Mariners': [' ', 'Mariners', ' ', ' ', '0', '0', '0.0', ' ', '7.7%', '15.1%', '22.8%'], 'Guardians': [' ', 'Guardians', ' ', ' ', '0', '0', '0.0', ' ', '7.5%', '7.6%', '15.2%'], 'Tigers': [' ', 'Tigers', ' ', ' ', '0', '0', '0.0', ' ', '5.9%', '6.3%', '12.1%'], 'Royals': [' ', 'Royals', ' ', ' ', '0'

In [11]:
def clean(total):
    df = pd.DataFrame(total[1:], columns=headers)
    df['game'] = df['game'].astype(int)
    
    # Add missing games
    new_rows = []
    grouped = df.groupby(['year', 'team'])
    
    for (year, team), group in grouped:
        existing_games = set(group['game'])
        min_game = min(existing_games)
        max_game = max(existing_games)
        
        for game in range(min_game, max_game + 1):
            if game not in existing_games:
                new_row = [year, team, game, ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
                new_rows.append(new_row)
    
    new_df = pd.DataFrame(new_rows, columns=headers)
    
    # Concatenate original and new rows
    df_updated = pd.concat([df, new_df], ignore_index=True)
    
    # Prioritize non-empty GBWC
    df_updated['GBWC'] = df_updated['GBWC'].fillna('')
    df_updated = df_updated.sort_values(by=['year', 'team', 'game', 'GBWC'], ascending=[True, True, True, False])
    
    # Remove duplicates, keeping the first occurrence (which has a non-empty GBWC if possible)
    df_updated = df_updated.drop_duplicates(subset=['year', 'team', 'game'], keep='first')
    
    df_updated = df_updated.sort_values(by=["year", "team", "game"]).reset_index(drop=True)

    return df_updated




In [12]:
def run(date, url_wc, url_lg, year):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    try:
        driver.get(url_wc)
        wc_data = load_page(driver, True, year)

        driver.get(url_lg)
        lg_data = load_page(driver, False, year)
        combined_data = combine(lg_data, wc_data, date)
        return list(combined_data.values())
    except Exception as e:
        print(f'{str(e)} Error on date: {date}')
        
    finally:
        driver.quit()

In [13]:
def start_reading(year, start, end, writeto, total = []):

    for i in range(start, end):
        df = clean(total)
        df.to_csv("L:/RA_work/JAY/datasets/final/2/" + str(writeto) + ".csv", index=False, header=headers)
        print(f"On {date_list[i]}")
        try:
            total.extend(run(date_list[i], url_wc[i], url_lg[i], year))
        except:
            continue
    df = clean(total)
    df.to_csv("L:/RA_work/JAY/datasets/final/2/" + str(writeto) + ".csv", index=False, header=headers)



In [25]:
def convert_progress(path) -> list:
    total = []
    with open(path, mode='r', newline='') as f:
        reader = csv.reader(f)
        
        for row in reader:
            total.append(row)
    return total

In [26]:
print(date_list[1459])
print(date_list[1495])

2018-03-28
2018-05-03


In [28]:
"""RUN THIS TO START"""

pre2018 = [250, 500, 750, 1000, 1250] # end at 1313
post2018 = [1500, 1750, 2000, 2250, 2500, 2750] # start at 1459, end at 2776
post2022 = [3000, 3250, 3500, 3750] # start at 2929, end at 3819

#total = []
#start_reading("2018", 1459, 1500, total)
# read up to end of 2017
#for i in pre2018:
    #total = []
    #start_reading("2017", i - 250, i, total)
#total = convert_progress("L:/RA_work/JAY/datasets/final/2/1250.csv")
#start_reading("2017", 1100, 1250, total)
#total = []
#start_reading("2017", 1250, 1313, total)

# read up to end of 2021
#total = convert_progress("L:/RA_work/JAY/datasets/final/2/1500.csv")
#start_reading("2018", 1491, 1500, total)
#post2018 = [2500, 2750]
#total = convert_progress("L:/RA_work/JAY/datasets/final/2/2250.csv")
#start_reading("2018", 2173, 2250, total)
#for i in post2018:
    #total = []
    #start_reading("2018", i - 250, i, total)
#total = convert_progress("L:/RA_work/JAY/datasets/final/2/2776.csv")

#start_reading("2018", 2752, 2776, total)
# read up to end of 2022
total = convert_progress("L:/RA_work/JAY/datasets/final/2/1500.csv")
start_reading("2018", 1459, 1460, 1500, total)


On 2018-03-28


In [17]:
print(date_list[2999])

2022-06-15


In [30]:
import glob

# Read all CSVs
all_files = glob.glob("L:/RA_work/JAY/datasets/final/2/*.csv")

# Merge all CSV files
df = pd.concat([pd.read_csv(f) for f in all_files])

# Apply the clean function
df_cleaned = clean(df.values.tolist())

# Save the cleaned and sorted CSV
df_cleaned.to_csv("L:/RA_work/JAY/datasets/final/merged_sorted_output.csv", index=False)